In [1]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import stats
from code.python.mpmath_integration import quad_phi
from mpmath import mp

from scipy import stats

In [120]:
x = mp.mpf('0.001')
alpha = mp.mpf('0.1')
beta = mp.mpf('0.01')
mu = mp.mpf('0')
delta = mp.mpf('1')


xmu = x - mu
gamma = mp.sqrt(alpha ** 2 - beta ** 2)
omega = mp.sqrt(xmu ** 2 + delta ** 2)

### 1. Expansion Phi

In [101]:
mp.dps = 100

In [91]:
a = xmu
b = -beta

In [92]:
def normcdf(x):
    return mp.erfc(-x / mp.sqrt(2)) / 2

In [93]:
t = 1/10

normcdf(a / mp.sqrt(t) + b * mp.sqrt(t))

mpf('0.9991279150062640223499260424132051641244775651490209495161703173045599120863914053546440630812454029394')

Expansion in terms of modified Bessel functions

In [83]:
N = 20

s = 0
for k in range(N):
    r = (-1) ** (k+1) * (b/a) ** k * mp.besselk(k + 1/2, a*b)
    s += r * t ** (1/2 + k)

1 + s * mp.exp(-a**2/(2*t) - b**2/2*t) / mp.pi * mp.sqrt(b / a)

mpc(real='1.548920819218252750919458960967515184906152621058019384639234987242424908450365957203423427104579076031', imag='0.0')

Expansion in terms of incomplete gamma function

In [84]:
N = 20

s = 0
for k in range(N):
    r = (-1) ** (k+1) / mp.factorial(k) * mp.gammainc(2*k + 1, a*b) / (a) ** (2*k + 1) * t**k / 2 ** (k + 1/2)
    s += r

mp.sqrt(t / mp.pi) * mp.exp(-a**2 / 2 / t) * s

mpf('0.5492692573537457611549574841954635175387391138185419098252932568023088277733111874610617537846795537958')

### 2. Expansion asymptotic $|x - \mu|$

In [121]:
mp_result = quad_phi(x, alpha, beta, mu, delta, digits=100)
mp_result

mpf('0.4918010466404691878489474923651672095276372189793093115002681717244902164560314258176381599885937580824')

In [122]:
xmu, alpha

(mpf('0.001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002'),
 mpf('0.1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002'))

In [58]:
mp.dps = 30
N = 22
C = delta * mp.exp(delta * gamma) / mp.sqrt(2 * mp.pi)

s = 0
for k in range(N):
    r = (-1) ** (k+1) / mp.factorial(k) / 2 ** (k + 1/2) * mp.gammainc(2*k + 1, -xmu * beta) / xmu ** (2*k + 1)
    # q = mp.quad(lambda t: t **(k-1) * mp.exp(-omega ** 2 / 2 / t - gamma ** 2 / 2 * t), [0, mp.inf])
    q = 2 * (omega / gamma) ** k * mp.besselk(k, gamma * omega)
    tk = r * q
    s += tk
    print(k, float(abs(tk)))

r1 = C * s / mp.sqrt(mp.pi)

0 2.147602025963213e-46
1 5.4027622682643735e-48
2 1.7845853759916656e-49
3 9.19214984212436e-51
4 6.671426918424911e-52
5 6.286987250100285e-53
6 7.313531838269578e-54
7 1.0154794283889663e-54
8 1.6431190471911143e-55
9 3.043178922476854e-56
10 6.361946094829941e-57
11 1.484614158442137e-57
12 3.832099987882878e-58
13 1.0858218554155037e-58
14 3.355716674719651e-59
15 1.1249206951290638e-59
16 4.0709146017296005e-60
17 1.5837178095184224e-60
18 6.598945218758981e-61
19 2.9353148843618054e-61
20 1.3897610065176596e-61
21 6.985208992933019e-62


In [59]:
float(abs(r1 / mp_result - 1))

1.8271528409914106e-16

In [60]:
mp.dps = 30
N = 22
C = delta * mp.exp(delta * gamma) / mp.sqrt(2 * mp.pi)

s = 0
for k in range(N):
    r = (-1) ** k / mp.factorial(k) * mp.gammainc(2*k + 1, -xmu * beta) * (omega / gamma / 2 / xmu ** 2) ** k * mp.besselk(k, gamma * omega)
    s += r

r2 = -delta * mp.exp(delta * gamma) / xmu / mp.pi * s

In [61]:
r1, r2

(mpf('3.49872348143932978837229854239407e-45'),
 mpf('3.49872348143933002753834119427445e-45'))

In [62]:
float(abs(r2 / mp_result - 1))

1.1435719752252184e-16

In [77]:
N = 22

gamma = mp.sqrt(alpha**2 - beta**2)
omega = mp.sqrt(xmu**2 + delta**2)

gw = gamma * omega

C = -delta * mp.exp(delta * gamma - gw) / mp.pi / xmu
z = -omega / gamma / xmu ** 2 / 2

k0 = mp.besselk(0, gw) * mp.exp(gw)
k1 = mp.besselk(1, gw) * mp.exp(gw)
rp = k0 / k1

u = -xmu * beta
y = mp.exp(-u)
u2 = u * u
v = 1
qp = y

t = C*k0

s = t * y
sp = s

for k in range(1, N):
    # Bessel ratio recursion
    r = 1.0 / rp + 2 * (k - 1) / gw;

    # Incomplete gamma recursion
    v *= u2
    m = 2*k * (2*k-1)
    q = m * qp + y * (v * (2*k / u + 1))
    
    t *= z * r / k
    s += t * q

    print(k, float(abs(t)), float(q), float(s))
    
    rp = r
    qp = q

float(s)

1 4.904664472972071e-47 1.8393972058572117 3.495886844203024e-45
2 1.2461990677542869e-49 23.912163676143752 3.498866775811144e-45
3 2.1320123616508263e-52 719.9400663725127 3.498713283699029e-45
4 2.7629158966794643e-55 40319.95463183125 3.498724423763389e-45
5 2.893000562484297e-58 3628799.9635386653 3.4987233739513554e-45
6 2.5495243213234596e-61 479001599.96953654 3.4987234960739785e-45
7 1.945054366246795e-64 87178291199.97385 3.498723479117327e-45
8 1.3113493438232842e-67 20922789887999.977 3.498723481861036e-45
9 7.9369827058441e-71 6402373705728000.0 3.49872348135288e-45
10 4.366508753242522e-74 2.43290200817664e+18 3.498723481459113e-45
11 2.2055452508672957e-77 1.1240007277776077e+21 3.498723481434323e-45
12 1.0313358848281418e-80 6.204484017332394e+23 3.498723481440722e-45
13 4.4958159015508505e-84 4.0329146112660565e+26 3.498723481438909e-45
14 1.8378642527508927e-87 3.0488834461171387e+29 3.498723481439469e-45
15 7.081590320295633e-91 2.6525285981219107e+32 3.4987234814392

3.49872348143933e-45

In [74]:
float(mp_result)

3.49872348143933e-45